In [4]:
import pandas as pd
import numpy as np
import datetime
from sklearn.svm import LinearSVR

In [5]:
train = pd.read_csv('all/train.csv')
test = pd.read_csv('all/test.csv')
nmt = pd.read_csv('all/new_merchant_transactions.csv')
mct = pd.read_csv('all/merchants.csv')
ht = pd.read_csv('all/historical_transactions.csv')

In [6]:
train = pd.read_csv('all/train.csv')


In [7]:
train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056
3,2017-09,C_ID_186d6a6901,4,3,0,0.142495
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159749


In [7]:
nmt.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_415bb3a509,107,N,1,B,307,M_ID_b0c793002c,1,-0.557574,2018-03-11 14:57:36,1.0,9,19
1,Y,C_ID_415bb3a509,140,N,1,B,307,M_ID_88920c89e8,1,-0.569580,2018-03-19 18:53:37,1.0,9,19
2,Y,C_ID_415bb3a509,330,N,1,B,507,M_ID_ad5237ef6b,2,-0.551037,2018-04-26 14:08:44,1.0,9,14
3,Y,C_ID_415bb3a509,-1,Y,1,B,661,M_ID_9e84cda3b1,1,-0.671925,2018-03-07 09:43:21,NaN,-1,8
4,Y,C_ID_ef55cf8d4b,-1,Y,1,B,166,M_ID_3c86fa3831,1,-0.659904,2018-03-22 21:07:53,NaN,-1,29


In [11]:
s = '2017-02'

In [12]:
a = s.split('-')

In [13]:
int(a[1])

2

In [14]:
train.shape

(201917, 6)

In [15]:
time = np.zeros((train.shape[0],1))

In [16]:
d1 = datetime.datetime.strptime(s,'%Y-%m')

In [17]:
d2 = datetime.datetime.strptime('2018-12','%Y-%m')

In [18]:
c = d2 - d1
int(c.days/30)

22

In [19]:
def cal_time(x):
    if isinstance(x['first_active_month'], str):
        d1 = datetime.datetime.strptime(x['first_active_month'],'%Y-%m')
        d2 = datetime.datetime.strptime('2018-12','%Y-%m')
        delta = d2 - d1
    else:
        return float('nan')

    
    return int(delta.days/30)


In [27]:
train['time'] = train.apply(cal_time,axis=1)

In [28]:
train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target,time
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283,18
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913,23
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056,28
3,2017-09,C_ID_186d6a6901,4,3,0,0.142495,15
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159749,13


In [29]:
train_X = train[['feature_1','feature_2','feature_3','time']]

In [30]:
train_y = train[['target']]

In [31]:
test['time'] = test.apply(cal_time,axis=1)

In [48]:
test_X = test[['feature_1','feature_2','feature_3','time']]

In [40]:
test_X.mean()

feature_1     3.109260
feature_2     1.741796
feature_3     0.564377
time         22.483708
dtype: float64

In [45]:
test.loc[11578,'time'] = 22

In [46]:
test.loc[11578,'time']

22.0

In [33]:
clf = LinearSVR(random_state=0, tol=1e-5)
clf.fit(train_X, train_y)
LinearSVR(C=1.0, dual=True, epsilon=0.0, fit_intercept=True,
     intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=1000,
     random_state=0, tol=1e-05, verbose=0)

/Users/ziluli/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/ziluli/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVR(C=1.0, dual=True, epsilon=0.0, fit_intercept=True,
     intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=1000,
     random_state=0, tol=1e-05, verbose=0)

In [34]:
clf.score(train_X,train_y)

-0.06945117938800238

In [49]:
y_pred = clf.predict(test_X)

In [50]:
y_pred

array([0.5243236 , 0.59303639, 0.4245973 , ..., 0.64731566, 0.46016304,
       0.62706853])

# Light GBM

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import lightgbm as lgb

In [ ]:
lgb_params = {"objective" : "regression", "metric" : "rmse", 
               "max_depth": 7, "min_child_samples": 20, 
               "reg_alpha": 1, "reg_lambda": 1,
               "num_leaves" : 64, "learning_rate" : 0.01, 
               "subsample" : 0.8, "colsample_bytree" : 0.8, 
               "verbosity": -1}

In [52]:
sample_submission = pd.read_csv('all/sample_submission.csv')
sample_submission['target'] = y_pred
sample_submission.to_csv('stacker_rcv.csv', index=False)

In [53]:
sample_submission.head()

,card_id,target
0,C_ID_0ab67a22ab,0.524324
1,C_ID_130fd0cbdd,0.593036
2,C_ID_b709037bc5,0.424597
3,C_ID_d27d835a9f,0.338680
4,C_ID_2b5e3df5c2,0.829540
